In [ ]:
# =====================================================
# Pegos Twitter Scraper (Final - DOM Fixed)
# =====================================================
import os, time, random
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def safe_int(val):
    if not val: return 0
    val = val.replace(",", "").strip()
    try:
        if val.endswith("B"):
            return int(float(val[:-1]) * 1000)
        elif val.endswith("M") or val.endswith("Mn"):
            return int(float(val[:-1]) * 1_000_000)
        return int(float(val))
    except:
        return 0

In [ ]:
AUTH_TOKEN = os.getenv("AUTH_TOKEN")
CT0 = os.getenv("CT0")
OUT_PATH = "/tmp/pegos_output.csv"

if not AUTH_TOKEN or not CT0:
    raise RuntimeError("AUTH_TOKEN veya CT0 tanımlı değil (GitHub Secrets kısmına ekle).")

opts = Options()
opts.add_argument("--headless=chrome")
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-dev-shm-usage")
opts.add_argument("--disable-gpu")
opts.add_argument("--window-size=1920,1080")
opts.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

In [ ]:
driver.get("https://x.com")
time.sleep(3)
driver.add_cookie({"name": "auth_token", "value": AUTH_TOKEN, "domain": ".x.com"})
driver.add_cookie({"name": "ct0", "value": CT0, "domain": ".x.com"})
driver.refresh()
time.sleep(5)
print("✅ Login başarılı, URL:", driver.current_url)

In [ ]:
KEYWORDS = ['blockchain', 'cryptocurrency', 'bitcoin']
tweetArr = []

for kw in KEYWORDS:
    print(f"🔎 Searching for: {kw}")
    driver.get(f"https://x.com/search?q={kw}&src=typed_query&f=live")
    time.sleep(5)

    for scroll_round in range(40):
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2.5, 3.8))
        driver.execute_script("window.scrollBy(0, -200);")

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        articles = soup.find_all("article") or []

        for art in articles:
            try:
                tag = art.find(attrs={"data-testid": "tweetText"})
                if not tag:
                    continue

                text = tag.get_text(" ", strip=True)
                time_tag = art.find("time")
                time_str = time_tag["datetime"] if time_tag else None

                # 🔹 Yeni DOM yapılarına göre değerleri al
                reply = art.find(attrs={"data-testid": "reply"})
                retweet = art.find(attrs={"data-testid": "retweet"})
                like = art.find(attrs={"data-testid": "like"})
                view = art.find(attrs={"data-testid": "viewCount"})

                reply_val = safe_int(reply.get_text(strip=True) if reply else "0")
                retweet_val = safe_int(retweet.get_text(strip=True) if retweet else "0")
                like_val = safe_int(like.get_text(strip=True) if like else "0")
                view_val = safe_int(view.get_text(strip=True) if view else "0")

                tweetArr.append({
                    "keyword": kw,
                    "tweet": text,
                    "time": time_str,
                    "comment": reply_val,
                    "retweet": retweet_val,
                    "like": like_val,
                    "see_count": view_val
                })
            except Exception as e:
                continue

    print(f"✅ Completed {kw}: {len(tweetArr)} total tweets so far")

driver.quit()

In [ ]:
df = pd.DataFrame(tweetArr)
if df.empty:
    print("⚠️ No tweets collected.")
else:
    df.drop_duplicates(subset=['tweet', 'time'], inplace=True)
    df.to_csv(OUT_PATH, index=False)
    print(f"💾 Saved to {OUT_PATH}, total {len(df)} rows.")